In [ ]:
import pandas as pd
import numpy as np  # 确保已导入numpy
import os

CONFIG = {
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "raw_data_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_result_path": r'./backtest_stable.csv',
    "backtest_summary_path": r'./backtest_stable_summary.txt',
    "log_path": r'./backtest_stable_log.txt',
    "trade_rule": {
        "buy_delay": 1,    # T+1买入
        "sell_delay": 5,   # T+5卖出
        "min_valid_days": 6
    }
}

# --------------------------
# 工具函数（核心修复：替换round为np.round）
# --------------------------
def init_environment():
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【稳定版回测启动】{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg("✅ 回测环境初始化完成")

def log_msg(msg):
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def calc_group_stats(df):
    """修复：用np.round兼容float类型，解决round报错"""
    # 1. 处理竞价得分分组
    if 'auction_score' in df.columns:
        try:
            df['auction_group'] = pd.qcut(
                df['auction_score'], 
                q=3, 
                labels=['低竞价得分', '中竞价得分', '高竞价得分'],
                duplicates='drop'
            )
        except:
            df['auction_group'] = pd.cut(
                df['auction_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低竞价得分', '中竞价得分', '高竞价得分']
            )
    else:
        df['auction_group'] = '无数据'
    
    # 2. 处理量价得分分组
    if 'price_volume_score' in df.columns:
        try:
            df['pv_group'] = pd.qcut(
                df['price_volume_score'],
                q=3,
                labels=['低量价得分', '中量价得分', '高量价得分'],
                duplicates='drop'
            )
        except:
            df['pv_group'] = pd.cut(
                df['price_volume_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低量价得分', '中量价得分', '高量价得分']
            )
    else:
        df['pv_group'] = '无数据'
    
    # 3. 分组统计（核心修复：用np.round替代round）
    group_stats = []
    # 竞价得分分组统计
    auction_stats = df.groupby('auction_group', observed=True).agg({
        # 替换x.mean().round(2) → np.round(x.mean(), 2)
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),  # 修复：用np.round兼容float
            lambda x: np.round(x.median(), 2),  # 修复
            lambda x: np.round((x>0).mean()*100, 2)  # 修复
        ],
        'stock_code': lambda x: x.nunique()
    })
    auction_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in auction_stats.index:
        group_stats.append({
            '分组类型': '竞价得分',
            '分组': group,
            '交易数': auction_stats.loc[group, '交易数'],
            '平均收益(%)': auction_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': auction_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': auction_stats.loc[group, '正收益比例(%)']
        })
    
    # 量价得分分组统计（同逻辑修复）
    pv_stats = df.groupby('pv_group', observed=True).agg({
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),  # 修复
            lambda x: np.round(x.median(), 2),  # 修复
            lambda x: np.round((x>0).mean()*100, 2)  # 修复
        ],
        'stock_code': lambda x: x.nunique()
    })
    pv_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in pv_stats.index:
        group_stats.append({
            '分组类型': '量价得分',
            '分组': group,
            '交易数': pv_stats.loc[group, '交易数'],
            '平均收益(%)': pv_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': pv_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': pv_stats.loc[group, '正收益比例(%)']
        })
    
    return pd.DataFrame(group_stats), df

def save_summary(summary_dict, group_stats_df):
    """保存汇总结果"""
    group_text = "\n4. 分组统计\n"
    group_text += "="*60 + "\n"
    group_text += group_stats_df.to_string(index=False, na_rep='-') + "\n"
    
    # 汇总内容中的数值也用np.round统一精度
    summary_content = f"""
【稳定版回测结果】
==========================
回测规则：T日选股 → T+{CONFIG['trade_rule']['buy_delay']}买入 → T+{CONFIG['trade_rule']['sell_delay']}卖出
==========================
1. 基础统计
   - 选股总记录：{summary_dict['total_selection']} 条
   - 有效交易：{summary_dict['valid_trade']} 条
   - 无效交易：{summary_dict['invalid_trade']} 条
   - 有效率：{np.round(summary_dict['valid_rate'], 2)}%

2. 收益统计
   - 平均收益率：{np.round(summary_dict['avg_return'], 2)}%
   - 中位数收益率：{np.round(summary_dict['median_return'], 2)}%
   - 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%（{summary_dict['positive_count']}/{summary_dict['valid_trade']}）
   - 最大收益：{np.round(summary_dict['max_return'], 2)}%
   - 最小收益：{np.round(summary_dict['min_return'], 2)}%

3. 风险统计
   - 收益标准差：{np.round(summary_dict['std_return'], 2)}%
   - 最大回撤：{np.round(summary_dict['max_drawdown'], 2)}%（简化计算）
{group_text}
=========================="""
    with open(CONFIG["backtest_summary_path"], 'w', encoding='utf-8') as f:
        f.write(summary_content)
    log_msg(f"✅ 回测汇总保存：{CONFIG['backtest_summary_path']}")

# --------------------------
# 主回测逻辑（保持不变）
# --------------------------
def run_backtest():
    try:
        init_environment()
        
        # 1. 加载数据
        log_msg("加载选股结果...")
        selection_df = pd.read_csv(CONFIG["selection_result_path"])
        selection_df['date'] = pd.to_datetime(selection_df['date']).dt.date
        log_msg(f"✅ 选股结果：{len(selection_df)}条记录，{selection_df['stock_code'].nunique()}只股票")
        
        log_msg("加载行情数据...")
        raw_df = pd.read_parquet(CONFIG["raw_data_path"])
        raw_df['date'] = pd.to_datetime(raw_df['date']).dt.date
        raw_df = raw_df[['stock_code', 'date', 'close']].dropna(subset=['close'])
        raw_df = raw_df.sort_values(['stock_code', 'date']).reset_index(drop=True)
        
        # 2. 匹配买卖价格
        log_msg("匹配买卖价格...")
        raw_df['trade_seq'] = raw_df.groupby('stock_code').cumcount()
        selection_df = selection_df.merge(
            raw_df[['stock_code', 'date', 'trade_seq']],
            on=['stock_code', 'date'],
            how='left'
        ).dropna(subset=['trade_seq'])
        selection_df['trade_seq'] = selection_df['trade_seq'].astype(int)
        
        buy_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['buy_delay']
        sell_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['sell_delay']
        
        buy_price = raw_df.set_index(['stock_code', 'trade_seq'])['close'].reindex(
            pd.MultiIndex.from_arrays([selection_df['stock_code'], buy_seq], names=['stock_code', 'trade_seq'])
        ).values
        selection_df['buy_price'] = buy_price
        
        sell_price = raw_df.set_index(['stock_code', 'trade_seq'])['close'].reindex(
            pd.MultiIndex.from_arrays([selection_df['stock_code'], sell_seq], names=['stock_code', 'trade_seq'])
        ).values
        selection_df['sell_price'] = sell_price
        
        # 3. 计算收益
        log_msg("计算收益...")
        selection_df['return_rate'] = (selection_df['sell_price'] - selection_df['buy_price']) / \
                                    selection_df['buy_price'].replace(0, 0.0001) * 100
        valid_mask = selection_df['buy_price'].notna() & selection_df['sell_price'].notna()
        backtest_result = selection_df[valid_mask].copy()
        invalid_count = len(selection_df) - len(backtest_result)
        
        # 4. 分组统计（修复后可正常执行）
        group_stats_df, backtest_result_with_group = calc_group_stats(backtest_result)
        
        # 5. 汇总统计
        if len(backtest_result) > 0:
            summary_dict = {
                "total_selection": len(selection_df),
                "valid_trade": len(backtest_result),
                "invalid_trade": invalid_count,
                "valid_rate": len(backtest_result)/len(selection_df)*100,
                "avg_return": backtest_result['return_rate'].mean(),
                "median_return": backtest_result['return_rate'].median(),
                "positive_count": (backtest_result['return_rate']>0).sum(),
                "positive_ratio": (backtest_result['return_rate']>0).mean()*100,
                "max_return": backtest_result['return_rate'].max(),
                "min_return": backtest_result['return_rate'].min(),
                "std_return": backtest_result['return_rate'].std(),
                "max_drawdown": 0
            }
        else:
            summary_dict = {k: 0 for k in ["total_selection", "valid_trade", "invalid_trade", "valid_rate", "avg_return", "median_return", "positive_count", "positive_ratio", "max_return", "min_return", "std_return", "max_drawdown"]}
        
        # 6. 保存结果
        backtest_result_with_group.to_csv(CONFIG["backtest_result_path"], index=False, encoding='utf-8-sig')
        save_summary(summary_dict, group_stats_df)
        
        # 打印结果
        log_msg(f"\n" + "="*60)
        log_msg(f"✅ 稳定版回测完成！核心结果：")
        log_msg(f"📊 有效交易：{summary_dict['valid_trade']}条 | 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%")
        log_msg(f"📈 平均收益：{np.round(summary_dict['avg_return'], 2)}% | 中位数收益：{np.round(summary_dict['median_return'], 2)}%")
        log_msg(f"📁 明细：{CONFIG['backtest_result_path']} | 汇总：{CONFIG['backtest_summary_path']}")
        log_msg("="*60)
    
    except Exception as e:
        log_msg(f"❌ 回测失败：{str(e)}")
        raise

if __name__ == "__main__":
    run_backtest()

[2025-10-24 13:07:21] ✅ 回测环境初始化完成
[2025-10-24 13:07:21] 加载选股结果...
[2025-10-24 13:07:21] ✅ 选股结果：104条记录，101只股票
[2025-10-24 13:07:21] 加载行情数据...
[2025-10-24 13:07:24] 匹配买卖价格...
[2025-10-24 13:07:25] 计算收益...
[2025-10-24 13:07:25] ❌ 回测失败：'float' object has no attribute 'round'


AttributeError: 'float' object has no attribute 'round'